In [2]:
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import sys
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

df = pd.read_csv('historical_data1_Q12005_clean.csv')
df2 = pd.read_csv('historical_data1_Q22005_clean.csv')

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# NOTE: DOES NOT USE mortgage_insurance_perc
dummies = pd.get_dummies(df[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y = df.orig_interest_rate
X_ = df.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X = pd.concat([X_, dummies[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R', 'channel_T',
                           'prepayment_penalty_mortgage_flag_Y', 'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)

In [4]:
# NOTE: DOES NOT USE mortgage_insurance_perc
dummies2 = pd.get_dummies(df2[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y_val = df2.orig_interest_rate
X_val_ = df2.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X_val = pd.concat([X_val_, dummies2[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R', 'channel_T',
                           'prepayment_penalty_mortgage_flag_Y', 'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)

In [5]:
def processSubset(feature_set):
# Fit model on feature_set and calculate RSS
    model = linear_model.LinearRegression()
    model.fit(X[list(feature_set)], y)
    
    #MAE, RMS, MAPE
    RMS_train = mean_squared_error(y, model.predict(X[list(feature_set)]))
    RMS_test = mean_squared_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAE_train = mean_absolute_error(y, model.predict(X[list(feature_set)]))
    MAE_test = mean_absolute_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAPE_train = np.mean(np.abs((y - model.predict(X[list(feature_set)])) / y)) * 100
    MAPE_test = np.mean(np.abs((y_val - model.predict(X_val[list(feature_set)])) / y_val)) * 100
    
    
    RSS = ((model.predict(X_val[list(feature_set)]) - y_val) ** 2).sum()
    return {"model":model, "RSS":RSS, "feature":X[list(feature_set)], 
            "RMS_train": RMS_train, "RMS_test": RMS_test,
            "MAE_train": MAE_train, "MAE_test": MAE_test,
            "MAPE_train": MAPE_train, "MAPE_test": MAPE_test}

In [ ]:
def exhaustive(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
        
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the smallest RMS_test
    best_model = models.loc[models['RMS_test'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.") # Return the best model, along with some other useful information about the model
    return best_model

# Could take quite awhile to complete...
models_exhaustive = pd.DataFrame(columns=["RSS", "model", "feature" , "RMS_train", 
                                "RMS_test", "MAE_train", "MAE_test", "MAPE_train", "MAPE_test"])
tic = time.time()
# exhaustive(1)
for i in range(1,10):
     models_exhaustive.loc[i] = exhaustive(i)
    
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

('Processed ', 19, 'models on', 1, 'predictors in', 0.7324609756469727, 'seconds.')
('Processed ', 19, 'models on', 1, 'predictors in', 0.722681999206543, 'seconds.')
('Processed ', 171, 'models on', 2, 'predictors in', 8.290249109268188, 'seconds.')
('Processed ', 969, 'models on', 3, 'predictors in', 69.25289607048035, 'seconds.')


In [6]:
def backward(predictors):
    tic = time.time()
    results = []
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo)) 
        # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RMS_test
    best_model = models.loc[models['RMS_test'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.") 
    # Return the best model, along with some other useful information about the model
    return best_model

models_backward = pd.DataFrame(columns=["RSS", "model", "feature", "RMS_train", 
                                "RMS_test", "MAE_train", "MAE_test", "MAPE_train", "MAPE_test"], index = range(1,len(X.columns)))
tic = time.time()
predictors = X.columns
while(len(predictors) > 1):
    models_backward.loc[len(predictors)-1] = backward(predictors)
    predictors = list(models_backward.loc[len(predictors)-1]["feature"])

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

('Processed ', 19, 'models on', 18, 'predictors in', 11.738043069839478, 'seconds.')
('Processed ', 18, 'models on', 17, 'predictors in', 7.785224914550781, 'seconds.')
('Processed ', 17, 'models on', 16, 'predictors in', 6.759774923324585, 'seconds.')
('Processed ', 16, 'models on', 15, 'predictors in', 6.168873071670532, 'seconds.')
('Processed ', 15, 'models on', 14, 'predictors in', 4.356022119522095, 'seconds.')
('Processed ', 14, 'models on', 13, 'predictors in', 4.108366012573242, 'seconds.')
('Processed ', 13, 'models on', 12, 'predictors in', 3.188616991043091, 'seconds.')
('Processed ', 12, 'models on', 11, 'predictors in', 2.6408450603485107, 'seconds.')
('Processed ', 11, 'models on', 10, 'predictors in', 2.267781972885132, 'seconds.')
('Processed ', 10, 'models on', 9, 'predictors in', 2.0139429569244385, 'seconds.')
('Processed ', 9, 'models on', 8, 'predictors in', 1.3922829627990723, 'seconds.')
('Processed ', 8, 'models on', 7, 'predictors in', 1.0812361240386963, 'sec

In [7]:
def forward(predictors):
    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
        
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the highest RSS
    best_model = models.loc[models['RMS_test'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    # Return the best model, along with some other useful information about the model
    return best_model

models_forward = pd.DataFrame(columns=["RSS", "model", "feature", "RMS_train", 
                                "RMS_test", "MAE_train", "MAE_test", "MAPE_train", "MAPE_test"])
tic = time.time()
predictors = []
for i in range(1,len(X.columns)+1):
    models_forward.loc[i] = forward(predictors)
    predictors = list(models_forward.loc[i]["feature"])
    
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

('Processed ', 19, 'models on', 1, 'predictors in', 0.7077529430389404, 'seconds.')
('Processed ', 18, 'models on', 2, 'predictors in', 1.0745420455932617, 'seconds.')
('Processed ', 17, 'models on', 3, 'predictors in', 1.2459800243377686, 'seconds.')
('Processed ', 16, 'models on', 4, 'predictors in', 1.411926031112671, 'seconds.')
('Processed ', 15, 'models on', 5, 'predictors in', 1.5811090469360352, 'seconds.')
('Processed ', 14, 'models on', 6, 'predictors in', 1.6902880668640137, 'seconds.')
('Processed ', 13, 'models on', 7, 'predictors in', 1.7815799713134766, 'seconds.')
('Processed ', 12, 'models on', 8, 'predictors in', 2.058013916015625, 'seconds.')
('Processed ', 11, 'models on', 9, 'predictors in', 2.0385661125183105, 'seconds.')
('Processed ', 10, 'models on', 10, 'predictors in', 2.1140880584716797, 'seconds.')
('Processed ', 9, 'models on', 11, 'predictors in', 2.253514051437378, 'seconds.')
('Processed ', 8, 'models on', 12, 'predictors in', 2.360352039337158, 'second

In [8]:
models_backward

,RSS,model,feature,RMS_train,RMS_test,MAE_train,MAE_test,MAPE_train,MAPE_test
1,54751.2,"LinearRegression(copy_X=True, fit_intercept=Tr...",orig_loantovalue 0 5...,0.126532,0.134968,0.271806,0.283533,4.84174,4.82862
2,53752.2,"LinearRegression(copy_X=True, fit_intercept=Tr...",orig_upb orig_loantovalue 0 190...,0.125003,0.132505,0.269747,0.281053,4.80767,4.78628
3,53040.9,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.122173,0.130752,0.266561,0.279768,4.75147,4.76388
4,52426.7,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.119902,0.129238,0.264696,0.279325,4.72017,4.75883
5,51829.9,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.117807,0.127767,0.262728,0.278475,4.68671,4.74506
6,51562,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.117512,0.127106,0.261804,0.277485,4.67055,4.72855
7,51369.8,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.117421,0.126632,0.261567,0.27675,4.66626,4.71687
8,51218.8,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.117129,0.12626,0.261591,0.27678,4.66699,4.7183
9,51097.2,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_upb orig_loantoval...,0.116444,0.12596,0.260642,0.276454,4.65009,4.71303
10,51008.8,"LinearRegression(copy_X=True, fit_intercept=Tr...",credit_score orig_debttoincome orig_...,0.116322,0.125742,0.260452,0.276163,4.64656,4.70795


In [9]:
def getBestModel(models):
    length = len(models.index)
    bestModel = models.loc[1]
    RMS = models.loc[1]["RMS_test"]
    for i in range (1, length + 1):
        if models.loc[i]["RMS_test"] < RMS:
            bestModel = models.loc[i]
            RMS = models.loc[i]["RMS_test"]
    return bestModel

In [10]:
bestmodel_backward = getBestModel(models_backward)
bestmodel_forward = getBestModel(models_forward)
print (bestmodel_backward["RMS_test"])
print (bestmodel_forward["RMS_test"])

0.125644420992
0.125637704592


In [12]:
print list(bestmodel_forward["feature"])

['orig_loantovalue', 'occupancy_status_I', 'credit_score', 'orig_upb', 'channel_T', 'loan_purpose_N', 'channel_R', 'no_borrower', 'orig_debttoincome', 'occupancy_status_S', 'first_time_homebuyer_flag_Y', 'prepayment_penalty_mortgage_flag_Y', 'loan_purpose_P', 'no_unit', 'occupancy_status_O', 'loan_purpose_C']
